In [5]:
import z from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "@langchain/openai";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";
import { ChatPromptTemplate } from "langchain/prompts";
import { load } from "dotenv";
const env = await load();
const process = { env };
const taggingSchema = z.object({ emotion: z.enum(["pos", "neg", "neutral"]).describe("文本的情感"), language: z.string().describe("文本的核心语言（应为ISO 639-1代码）") });
const model = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
    temperature: 0,
  },
});
const modelTagging = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "tagging",
        description: "为特定的文本片段打上标签",
        parameters: zodToJsonSchema(taggingSchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: { name: "tagging" },
  },
});
const prompt = ChatPromptTemplate.fromMessages([
  ["system", "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记"],
  ["human", "{input}"],
]);
const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser());
let res = await chain.invoke({
  input: "写代码太难了，爷不干了！",
});
console.log(res);


TypeError: array.map is not a function